## Load Libraries

In [1]:
import re
import random
import string
from datetime import datetime
from progressbar import ProgressBar
import pandas as pd
pbar = ProgressBar()
import collections
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd
import os
import glob
import seaborn as sns
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora, models, similarities, matutils
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
import numpy as np
import pickle
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.en.stop_words import STOP_WORDS

#import nltk
#nltk.download()

In [2]:
from langdetect import detect 
from textblob import TextBlob
from langdetect import detect_langs

In [131]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

## Merge

In [ ]:
os.chdir("csv")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "review_set.csv", index=False, encoding='utf-8-sig')
os.chdir("..")
#print(os.getcwd())

## Load

In [435]:
def lang(sentence):
    try:
        return detect(sentence)
    except:
        None

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    manual_stop = ['bumble', 'tinder', 'zoosks', 'zoosk', 'bumble', 'east meets east', 'grindr', 'hinge',
                  'cmb', 'okcupid','bumble', 'tinders', 'zoosks', 'zoosk', 'bumbles', 'east meets east', 'grindrs', 'hinges',
                  'cmbs', 'okcupids']
    stop.extend(manual_stop)
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

def clean_again_text(text):
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove stop words
    stop = stopwords.words('english')
    manual_stop = ['bumble', 'tinder', 'zoosks', 'zoosk', 'bumble', 'east meets east', 'grindr', 'hinge','okc',
                  'cmb', 'okcupid','bumble', 'tinders', 'zoosks', 'zoosk', 'bumbles', 'east meets east', 'grindrs', 'hinges',
                  'cmbs', 'okcupids', 'im', 'i','think','ive', 'want', 'lol', 'haha' , 'cupid', 'ok', 'let', 'know',
                  'right','dont','lot','happn','league','blk','jswipe','hi','cool','need','wont','need','bumblesupport','youre','apps']
    stop.extend(manual_stop)
    stop.extend(STOP_WORDS)
    
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)




In [218]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
#reviews = pd.read_csv('csv/review_set.csv')
n = 100 
reviews = pd.read_csv('csv/review_set.csv', header=0#, skiprows=lambda i: i % n != 0
                     )
reviews.columns = ['app_review_id','app', 'time', 'stars', 'title', 'review']
#reviews.to_csv( "review_set_removed_stop.csv", index=False, encoding='utf-8-sig')
#reviews = pd.read_csv('csv/review_set_removed_stop.csv')

In [ ]:
reviews['lang'] = reviews['review'].apply(lambda x: lang(x))
reviews.drop(reviews[reviews['lang'] != 'en'].index, inplace = True)

In [436]:
# clean text data
#reviews["review_clean"] = reviews["review"].apply(lambda x: clean_text(x))
#reviews['review_clean'] = reviews['review_clean'].apply(lambda x:  re.sub('([^\x00-\x7F])+', '', x))
#reviews['review_clean'] = reviews['review_clean'].apply(lambda x: re.sub(r"[\/\-\'\)\(,.;@#?!&$]+\ *", " ", x))
reviews["review_clean"] = reviews['review_clean'].apply(lambda x: clean_again_text(x))

In [22]:
pickle.dump(reviews, open("reviews_cleaned_1", "wb"))

In [3]:
reviews = pickle.load( open("reviews_cleaned_1", "rb"))
reviews.reset_index(inplace=True)

## PCA

In [ ]:
from sklearn.feature_extraction.text import  TfidfVectorizer 
from sklearn.decomposition import PCA

In [ ]:
tfidf = TfidfVectorizer()

# note you can also set n_components directly here if you want
# a lower dimensional projection
pca = PCA()

X = tfidf.fit_transform(reviews['review_clean'])
X = pca.fit_transform(X.toarray())

In [ ]:
def show_variance_explained_plots(pca):
    
    var_exp_array = pca.explained_variance_ratio_
    n_comps = var_exp_array.shape[0] 
    
    fig, ax = plt.subplots(1,2,figsize=(10,4))
    
    ax[0].fill_between(range(n_comps), var_exp_array)
    ax[0].set_title('Variance Explained by Nth Principal Component')
    
    ax[1].fill_between(range(n_comps), np.cumsum(var_exp_array))
    ax[1].set_title('Cumulative Variance Explained by N Components')
    
    plt.show()

In [ ]:
show_variance_explained_plots(pca)

## LDA

In [ ]:
# Create a CountVectorizer for parsing/counting words
count_vectorizer = CountVectorizer(ngram_range=(1, 2), min_df = 20, max_df = .1,
                                   stop_words='english', token_pattern="\\b[a-z][a-z]+\\b")

count_vectorizer.fit(reviews['review_clean'])

doc_word = count_vectorizer.transform(reviews['review_clean']).transpose()


#min df and max df
#more stop words
#more topics

In [ ]:
doc_word.shape

In [ ]:
pd.DataFrame(doc_word.toarray(), count_vectorizer.get_feature_names())

In [ ]:
# Convert sparse matrix of counts to a gensim corpus
corpus = matutils.Sparse2Corpus(doc_word)
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())
# Create lda model (equivalent to "fit" in sklearn)
lda = models.LdaModel(corpus=corpus, num_topics=10, id2word=id2word, passes=5)
lda_corpus = lda[corpus]
lda_docs = [doc for doc in lda_corpus]


In [ ]:
lda.print_topics()


In [ ]:
lda_docs[:10]

## LDA with TFIDF

In [13]:
# Create a CountVectorizer for parsing/counting words
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df = 30, max_df = .1 ,#min_df = .01, max_df = .2,
                                   stop_words='english', token_pattern="\\b[a-z][a-z]+\\b")

tfidf_vectorizer.fit(reviews['review_clean'])

doc_word = tfidf_vectorizer.transform(reviews['review_clean']).transpose()


In [14]:
# Convert sparse matrix of counts to a gensim corpus
corpus = matutils.Sparse2Corpus(doc_word)
id2word = dict((v, k) for k, v in tfidf_vectorizer.vocabulary_.items())
# Create lda model (equivalent to "fit" in sklearn)
lda = models.LdaModel(corpus=corpus, num_topics=10, id2word=id2word, passes=10)
lda_corpus = lda[corpus]
lda_docs = [doc for doc in lda_corpus]


In [15]:
pd.DataFrame(doc_word.toarray(), tfidf_vectorizer.get_feature_names())

,0,1,2,3,4,5,6,7,8,9,...,91043,91044,91045,91046,91047,91048,91049,91050,91051,91052
abandon,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ability,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
able,0.065993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
able access,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
able app,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zero response,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zero star,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zip,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zone,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
lda.print_topics()

[(0,
  '0.022*"fake" + 0.018*"account" + 0.016*"facebook" + 0.011*"fake profile" + 0.011*"sign" + 0.009*"number" + 0.009*"scammer" + 0.007*"create" + 0.006*"phone" + 0.006*"phone number"'),
 (1,
  '0.146*"good app" + 0.019*"cool idea" + 0.019*"app crash" + 0.009*"dope" + 0.008*"io" + 0.007*"crash" + 0.007*"new version" + 0.007*"new update" + 0.006*"people come" + 0.006*"worth"'),
 (2,
  '0.011*"mile" + 0.009*"away" + 0.008*"mile away" + 0.008*"need" + 0.008*"woman" + 0.008*"look" + 0.008*"user" + 0.007*"distance" + 0.007*"men" + 0.006*"apps"'),
 (3,
  '0.099*"cool" + 0.056*"love" + 0.030*"fun" + 0.028*"concept" + 0.023*"love app" + 0.023*"easy" + 0.021*"charm" + 0.021*"nice" + 0.019*"easy use" + 0.018*"new"'),
 (4,
  '0.199*"great app" + 0.030*"best" + 0.026*"far" + 0.013*"app far" + 0.010*"best date" + 0.010*"battery" + 0.010*"meeting" + 0.009*"date app" + 0.009*"like concept" + 0.009*"meeting people"'),
 (5,
  '0.008*"swipe" + 0.006*"idea" + 0.006*"day" + 0.006*"person" + 0.005*"happ

## LDA with TFIDF App

## LSA

In [11]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words='english')
doc_word = vectorizer.fit_transform(reviews['review_clean'])
doc_word.shape

In [ ]:
pd.DataFrame(doc_word.toarray(), index=reviews['review_clean'], columns=vectorizer.get_feature_names()).head(10)


In [ ]:
lsa = TruncatedSVD(5)
doc_topic = lsa.fit_transform(doc_word)
lsa.explained_variance_ratio_


In [ ]:
topic_word = pd.DataFrame(lsa.components_.round(3),
             #index = ["component_1","component_2","component_3","component_4" ],
             columns = vectorizer.get_feature_names())

In [ ]:
display_topics(lsa, vectorizer.get_feature_names(), 10)


In [ ]:
doc_topic_lsa = pd.DataFrame(doc_topic.round(5),
                             index = reviews['review_clean'],
                            # columns = ["component_1","component_2","component_3","component_4" ]
                            )
doc_topic_lsa.head(10)

## LSA with TFIDF

In [ ]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df = .02, max_df = .1,
                                   stop_words='english')
doc_word = tfidf_vectorizer.fit_transform(reviews['review_clean'])
doc_word.shape

In [ ]:
lsa = TruncatedSVD(10)
doc_topic = lsa.fit_transform(doc_word)
lsa.explained_variance_ratio_


In [ ]:
topic_word = pd.DataFrame(lsa.components_.round(3),
             #index = ["component_1","component_2","component_3","component_4" ],
             columns = tfidf_vectorizer.get_feature_names())


In [ ]:
display_topics(lsa, tfidf_vectorizer.get_feature_names(), 10)


In [ ]:
doc_topic_lsa = pd.DataFrame(doc_topic.round(5),
                             index = reviews['review_clean'],
                            # columns = ["component_1","component_2","component_3","component_4" ]
                            )
doc_topic_lsa.head(10)

## NMF

In [5]:
vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words = 'english')
doc_word = vectorizer.fit_transform(reviews['review_clean'])


In [ ]:
pd.DataFrame(doc_word.toarray(), index=reviews['review_clean'], columns=vectorizer.get_feature_names()).head(10)


In [ ]:
nmf_model = NMF(5)
doc_topic = nmf_model.fit_transform(doc_word)

In [ ]:
topic_word = pd.DataFrame(nmf_model.components_.round(3),
             #index = ["component_1","component_2","component_3","component_4"],
             columns = vectorizer.get_feature_names())
topic_word

In [ ]:
display_topics(nmf_model, vectorizer.get_feature_names(), 10)

In [ ]:
doc_topic_nmf = pd.DataFrame(doc_topic.round(5),
                             index = reviews['review_clean'],
                             #columns = ["component_1","component_2","component_3","component_4"]
                            )
doc_topic_nmf

## NMF with TFIDF

In [459]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(0,1), min_df = 20, max_df = .1,
                                   stop_words = 'english')
doc_word = tfidf_vectorizer.fit_transform(reviews['review_clean'])


In [463]:
nmf_model = NMF(7)
doc_topic = nmf_model.fit_transform(doc_word)

In [471]:
topic_word = pd.DataFrame(nmf_model.components_.round(3),
             index = ["Customer_Support","Postive","Explanatory","Technical",
                     "UX_UI","Scam","Monetary"],
             columns = tfidf_vectorizer.get_feature_names())

topic_word

abandon  ability   able  abroad  absolute  absolutely  \
Customer_Support  0.001    0.011    0.249  0.006   0.025     0.194        
Postive           0.000    0.001    0.030  0.000   0.000     0.035        
Explanatory       0.016    0.101    0.424  0.003   0.038     0.104        
Technical         0.001    0.000    0.011  0.001   0.000     0.000        
UX_UI             0.000    0.008    0.000  0.000   0.000     0.000        
Scam              0.002    0.006    0.000  0.000   0.005     0.007        
Monetary          0.000    0.008    0.062  0.000   0.004     0.008        

                  absurd  abundance  abuse  abusive  accept  acceptable  \
Customer_Support  0.005   0.001      0.008  0.010    0.020   0.006        
Postive           0.000   0.000      0.000  0.000    0.002   0.000        
Explanatory       0.025   0.005      0.010  0.011    0.038   0.008        
Technical         0.000   0.000      0.000  0.000    0.001   0.001        
UX_UI             0.000   0.000      0.000  0.000    0.000   0.000        
Scam              0.000   0.000      0.001  0.003    0.000   0.001        
Monetary          0.008   0.000      0.000  0.000    0.009   0.000        

                  accepted  access  accessibility  accessible  accident  \
Customer_Support  0.000     0.235   0.001          0.003       0.011      
Postive           0.000     0.000   0.000          0.002       0.001      
Explanatory       0.006     0.059   0.003          0.008       0.025      
Technical         0.001     0.004   0.000          0.001       0.000      
UX_UI             0.000     0.017   0.004          0.008       0.004      
Scam              0.000     0.042   0.001          0.002       0.000      
Monetary          0.000     0.024   0.000          0.003       0.000      

                  accidental  accidentally  accomplish  accord  accordingly  \
Customer_Support  0.000       0.043         0.001       0.007   0.002         
Postive           0.000       0.000         0.000       0.000   0.000         
Explanatory       0.014       0.090         0.009       0.017   0.005         
Technical         0.000       0.000         0.000       0.002   0.000         
UX_UI             0.001       0.021         0.000       0.000   0.000         
Scam              0.000       0.000         0.002       0.003   0.000         
Monetary          0.000       0.000         0.000       0.000   0.000         

                  account  accounts   acct  accumulate  accuracy  accurate  \
Customer_Support  3.972    0.004     0.013  0.000       0.000     0.002      
Postive           0.000    0.000     0.000  0.000       0.000     0.000      
Explanatory       0.000    0.003     0.005  0.005       0.006     0.029      
Technical         0.000    0.000     0.000  0.000       0.001     0.001      
UX_UI             0.000    0.000     0.000  0.000       0.000     0.019      
Scam              0.000    0.000     0.002  0.000       0.000     0.002      
Monetary          0.000    0.000     0.003  0.001       0.000     0.000      

                  accurately  accuse  achieve  acknowledge  acquire    act  \
Customer_Support  0.000       0.013   0.000    0.009        0.002    0.035   
Postive           0.000       0.000   0.000    0.000        0.000    0.000   
Explanatory       0.006       0.002   0.004    0.008        0.005    0.024   
Technical         0.002       0.000   0.000    0.000        0.000    0.007   
UX_UI             0.000       0.000   0.000    0.000        0.000    0.000   
Scam              0.002       0.004   0.000    0.001        0.000    0.000   
Monetary          0.000       0.000   0.000    0.000        0.001    0.001   

                  action  activate  activation  active  actively  activity  \
Customer_Support  0.024   0.024     0.000       0.082   0.008     0.027      
Postive           0.000   0.000     0.000       0.000   0.000     0.000      
Explanatory       0.033   0.014     0.011       0.149   0.019     0.042      
Technical         0.

In [476]:
display_topics(nmf_model, tfidf_vectorizer.get_feature_names(), 25)


Topic  0
account, ban, log, delete, reason, new, facebook, phone, number, email, fix, service, customer, support, update, tell, sign, didnt, create, wrong, open, help, contact, issue, login

Topic  1
love, thank, life, concept, best, friend, awesome, new, recommend, help, thanks, idea, twitter, happy, experience, true, amaze, guy, team, connect, definitely, amazing, year, connection, facebook

Topic  2
money, look, swipe, way, woman, guy, day, waste, fake, month, person, send, girl, talk, thing, men, actually, bad, user, far, away, real, pretty, start, feature

Topic  3
work, update, crash, doesnt, open, fix, fine, notification, load, concept, iphone, phone, bug, issue, freeze, new, pretty, problem, idea, far, stop, reinstall, actually, screen, version

Topic  4
easy, fun, new, way, nice, navigate, simple, friend, far, recommend, pretty, best, super, awesome, interface, connect, quality, bagel, concept, friendly, enjoy, wish, meeting, online, definitely

Topic  5
site, best, nice, fak

In [477]:
doc_topic_nmf = pd.DataFrame(doc_topic.round(5),
                             index = reviews['review_clean'],
                             columns = ["Customer_Support","Postive","Explanatory","Technical",
                                         "UX_UI","Scam","Monetary"]
                            )

In [478]:
doc_topic_nmf.reset_index(inplace = True)
doc_topic_nmf

,review_clean,Customer_Support,Postive,Explanatory,Technical,UX_UI,Scam,Monetary
0,completely love app havent use date year ago decide try research decide try look app easy use wish chat window bit interactive allow delete message purpose app use step continue chatting texting match woman nice able step initiate conversation date nice guy find current boyfriend soon celebrate anniversary happy find app thing recommend change allow choose picture phone instead facebook meet guy old photo facebook didnt interact meet person look bit different overall good app recommend look guy nice educate couple absolute gentleman,0.00835,0.01534,0.04152,0.00000,0.01963,0.00083,0.00000
1,empowering woman app hard establish legitimate relationship require location service continually believe spy user require site result ton likes people live near setting match men mile radius inundate people visit city colossal waste time scroll people live state look hookup travel remember turn location swarm likes people country flaw substandard site like user set city live chooses radius view match conversation date local people hookup app change fan allow app constant access location time waste scrolling visitor delete soon fix flaw woman stick long woman fine find hookup app empower woman choice opt nonsense busy,0.00021,0.00000,0.03693,0.00150,0.00000,0.02264,0.00000
2,lemme start guy mad match low score app tip match cant today simply blurry picture party youll swipe try actually use bio link social platform female great time app great talk hour date hate match talk message plus feel like hotter guy app guy wanna date try relationship thing add man meet fianc marry june app work doesnt recycle guy lower standard guy meet college solely look guy yall college graduate meet love life thanks,0.00000,0.01647,0.03758,0.01400,0.00000,0.00000,0.00000
3,tried time find pattern men use gather woman hook upload great pic profile sit watch message pour woman initiate contact good look guy sit choice woman prey sexual encounter little men actually relationship oh half reply message lady work message lady men lazy spoil rotten attention cause knock door concept empower backfire men use advantage small percentage actually guy navigating heart look relationship thats date app decent look guy pick woman lay easy change woman message thing feature woman accept message guy sexes message empower woman feature woman use chose unlock view message guy hope change feature benefit dude believe,0.00000,0.00015,0.05058,0.00931,0.00830,0.00000,0.00000
4,great find date friend business partner app cant speak guys perspective app year absolute game changer love concept lady introvert come appreciate taught brave creative start conversation stranger like people genuine wish bit diversity love time limit concept match ingenious creates sense urgency great con app smooth user friendly plus limit swipe ice cake pay subscription allow swipe great perk sure easy waste time app swipe notify user cute little positive message tip love havent major problem app meet pretty people sleep yall,0.00000,0.04517,0.03125,0.00108,0.01918,0.00000,0.00024
...,...,...,...,...,...,...,...,...
91048,app like dislike try swipe way premium option answer,0.00000,0.00000,0.02855,0.00000,0.00456,0.00000,0.00000
91049,hate fact order new message etc close app reopen refresh,0.00782,0.00145,0.00606,0.00197,0.00818,0.00000,0.00000
91050,block feature work block profile pop,0.00037,0.00000,0.00594,0.05462,0.00000,0.00000,0.00147
91051,set distance app force thousand swipe mile radius update app trash impossible match like mile away,0.00000,0.00000,0.02406,0.00279,0.00000,0.00000,0.00000


In [479]:
df_doc = doc_topic_nmf

df_doc[['Customer_Support', 'Postive', 'Explanatory',
       'Technical', 'UX_UI', 'Scam', 'Monetary']] = df_doc[['Customer_Support', 'Postive', 'Explanatory',
       'Technical', 'UX_UI', 'Scam', 'Monetary']].apply(pd.to_numeric)

df_doc['topics'] = df_doc[['Customer_Support', 'Postive', 'Explanatory', 'Technical', 'UX_UI', 'Scam', 'Monetary']].idxmax(axis=1, skipna = True)

df_merged = df_doc.merge(reviews, how='inner', left_index=True, right_index=True, left_on=None, right_on=None)


## Visuals

In [510]:
topic_word_transposed = topic_word.transpose()
topic_word_transposed

,Customer_Support,Postive,Explanatory,Technical,UX_UI,Scam,Monetary
abandon,0.001,0.000,0.016,0.001,0.000,0.002,0.000
ability,0.011,0.001,0.101,0.000,0.008,0.006,0.008
able,0.249,0.030,0.424,0.011,0.000,0.000,0.062
abroad,0.006,0.000,0.003,0.001,0.000,0.000,0.000
absolute,0.025,0.000,0.038,0.000,0.000,0.005,0.004
...,...,...,...,...,...,...,...
yr,0.001,0.003,0.012,0.000,0.001,0.004,0.000
zero,0.084,0.000,0.124,0.000,0.000,0.009,0.010
zip,0.000,0.000,0.006,0.002,0.001,0.000,0.000
zone,0.000,0.000,0.010,0.001,0.003,0.000,0.000


In [509]:
df_merged.to_csv('visualize_me.csv')


In [511]:
topic_word_transposed.to_csv('topic_word_transposed.csv')


## APP Portion

In [380]:
# import distance methods
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import pairwise_distances

df_doc = doc_topic_nmf

def get_related_app(phrase):
    phrase = clean_text(phrase)
    phrase = re.sub('([^\x00-\x7F])+', '', phrase)
    phrase = re.sub(r"[\/\-\'\)\(,.;@#?!&$]+\ *", " ", phrase)
    phrase = clean_again_text(phrase)
    phrase = [phrase]
    phrase_doc_word = tfidf_vectorizer.transform(phrase)
    phrase_doc_topic = nmf_model.transform(phrase_doc_word)
    
    d = pairwise_distances(doc_word,phrase_doc_word,metric='cosine')
    #[films[m] for m in d[1].argsort()[:10]]
    df = pd.DataFrame(data=d, columns=["signal"])
    df_merged = df.merge(df_doc, how='inner', left_index=True, right_index=True, left_on=None, right_on=None)
    df_fin_merged = df_merged.merge(reviews, how='inner', left_index=True, right_index=True)
    _ = df_fin_merged[df_fin_merged['stars'] >= 3].sort_values('signal', ascending=True).head(5)
    _ = _.groupby('app')['signal'].mean().reset_index().sort_values('signal', ascending=True)
    return(translate_name(_['app'].iloc[0]))
    #translate_name(_.iloc[0:,[0]])
    # _ = _.groupby('app')['signal'].mean().reset_index()
    #_ = _.sort_values('signal', ascending=True).head(1)
    #return(_)

In [381]:
get_related_app('Hookup')

'Zoosk'

In [238]:
with open("doc_topic_nmf.pkl", "wb") as f:
    pickle.dump(doc_topic_nmf, f)

with open("tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(tfidf_vectorizer, f)

with open("nmf_model.pkl", "wb") as f:
    pickle.dump(nmf_model, f)

with open("reviews.pkl", "wb") as f:
    pickle.dump(reviews, f)
    
with open("doc_word.pkl", "wb") as f:
    pickle.dump(doc_word, f)

In [301]:
phrase = 'Asian'
phrase = clean_text(phrase)
phrase = re.sub('([^\x00-\x7F])+', '', phrase)
phrase = re.sub(r"[\/\-\'\)\(,.;@#?!&$]+\ *", " ", phrase)
phrase = clean_again_text(phrase)
phrase = [phrase]
phrase_doc_word = tfidf_vectorizer.transform(phrase)
phrase_doc_topic = nmf_model.transform(phrase_doc_word)

d = pairwise_distances(doc_word,phrase_doc_word,metric='cosine')
#[films[m] for m in d[1].argsort()[:10]]
df = pd.DataFrame(data=d, columns=["signal"])
df_merged = df.merge(df_doc, how='inner', left_index=True, right_index=True, left_on=None, right_on=None)
df_fin_merged = df_merged.merge(reviews, how='inner', left_index=True, right_index=True)
_ = df_fin_merged[df_fin_merged['stars'] >= 3].sort_values('signal', ascending=True).head(5)
_ = _.groupby('app')['signal'].mean().reset_index()
_ = _.sort_values('signal', ascending=True).head(1)
return(translate_name(_))
#.sort_values('signal', ascending=True).head(20)



SyntaxError: 'return' outside function (<ipython-input-301-0a390d7a3aba>, line 18)

In [310]:
def translate_name(app):
    return app_name_dict[app]
    

In [309]:
app_name_dict = {'tinder':'Tinder',
                 'cmb':'Coffee Meets Bagel',
                 'zoosk':'Zoosk',
                 'bumble':'Bumble',
                 'grindr':'Grindr',
                 'hinge':'Hinge',
                 'crispa':'Crispa',
                 'blk':'Black',
                 'match':'Match',
                 'east':'East Meets East',
                 'jswipe':'Jswipe',
                 'league':'The League',
                 'farmers':'FarmersOnly',
                 'happn':'Happn',
                 'match': 'Match.com'}

In [311]:
translate_name('tinder')

'Tinder'

In [307]:
reviews['app'].unique()

array(['bumble', 'jswipe', 'cmb', 'grindr', 'match', 'league', 'farmers',
       'tinder', 'zoosk', 'east', 'crispa', 'hinge', 'okcupid', 'happn',
       'blk'], dtype=object)